In [19]:
import pandas as pd
import matplotlib.pyplot

students_df = pd.read_csv('students_performance.csv')
print(students_df.head(5))

   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


In [20]:
 #info block
end_str = '\n' + '=' * 50 + '\n'

print(students_df.info(), end=end_str)

#строки столбцы
print(students_df.shape, end=end_str)
print(students_df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
None
(1000, 8)
        gender race/ethnicity parental level of education     lunch  \
count     1000           1000                        1000      1000   
unique       2              5                           6         2   
top     female        group C                s

In [21]:
#работа с столбцами переименовать в стиль питона
col = students_df.columns
col = col.str.replace(' ', '_').str.lower()
students_df.columns = col
students_df

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [22]:
#чистка от дубликатов и пропусков
if students_df.duplicated().any():
    print(f'🔁 Найдено {students_df.duplicated().sum()} дубликатов.')
    df = students_df.drop_duplicates().reset_index(drop=True)
    print('✅ Дубликаты удалены.')
else:
    print('👌 Дубликатов не найдено.')


👌 Дубликатов не найдено.


In [7]:
mask = students_df.reading_score == 100
students_df[mask]

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
108,female,group D,master's degree,standard,none,87,100,100
116,female,group E,bachelor's degree,standard,completed,99,100,100
151,male,group E,associate's degree,free/reduced,completed,100,100,93
167,female,group C,bachelor's degree,standard,completed,96,100,100
181,female,group D,some high school,standard,completed,97,100,100
383,male,group C,associate's degree,standard,completed,87,100,95
460,female,group E,bachelor's degree,standard,none,100,100,100
548,female,group A,some high school,standard,completed,92,100,97
568,female,group E,bachelor's degree,free/reduced,completed,92,100,100
596,female,group C,bachelor's degree,standard,completed,92,100,99


In [27]:
students_df.isna().sum()

np.int64(0)